# Analyzing your Datasets

This notebook will take you around the different ways you can analyze your data. 
This involves looking at basic metrics in the larger dataset, taking a random sample, creating derived variables, making sense of the missing values & so on. 

We will be mostly using SQL (via psycopg2), hence giving you the opportunity to interact with the database directly. The same queries can also be handled by Pandas in Python (by converting your datasets into dataframes). 

# Table of Contents

- [Introduction](#Introduction)
    - [Learning Objectives](#Learning-Objectives)
    - [Methods](#Methods)
- [Setup-Coding](#Setup-Coding)
    - [Imports](#Imports)
    - [Connection-Settings](#Connection-Settings)
    - [Rollback Coding](#Rollback-Coding)
    
- [Analysis](#Analysis)
    - [Step-1](#Step-1)
    - [STEP-2; DATA-A](#STEP-2;-DATA-A)
    - [STEP-2; DATA-B](#STEP-2;-DATA-B)
    - [STEP-2; DATA-C](#STEP-2;-DATA-C)
    - [STEP-2; DATA-D](#STEP-2;-DATA-D)
    - [STEP-2; DATA-E-1](#STEP-2;-DATA-E-1)
    - [STEP-2; DATA-E-2](#STEP-2;-DATA-E-2)
    - [STEP-2; DATA-F](#STEP-2;-DATA-F)
    
- [Using summary queries in SQL](#Using-summary-queries-in-SQL)
- [Getting to know the IDHS database](#Getting-to-know-the-IDHS-database)
- [Printing by INDEX](#Printing-by-INDEX)
- [Sample Queries](#Sample-Queries)

## Introduction
- Back to [Table of Contents](#Table-of-Contents)

In an ideal world, we will have all of the data we want with all of the desirable properties (no missing values, no errors, standard formats, and so on). 
However, that is hardly ever true- and we have to work with using our datasets to answer questions of interest as intelligently as possible. 

In this notebook, we will use our datasets to answer some questions of interest. 

### Learning Objectives
- Back to [Table of Contents](#Table-of-Contents)

After going through this notebook, you will have a good understanding around: 

- How to create new tables of interest from the larger tables in database
- How to decide on the variables of interest
- How to quickly look through aggregate metrics before proceeding with analysis
- Possible pitfalls
- How to handle missing values
- How to join newly created tables
- How to think about caveats in your final results

### Methods
- Back to [Table of Contents](#Table-of-Contents)

We will be using the psycopg2 Python package to access tables in our class database  server - PostgreSQL. 

To read the results of our queries, we will be using the pandas Python package, which has the ability to read tabular data from SQL queries into a pandas DataFrame object.

Within SQL, we will use various queries:

- CREATE TABLE
- SELECT ROWS
- SUMMING OVER GROUPS
- Counting Distinct Values of desired variables
- Ordering data by chosen variables
- Selecting a random sub-sample

### Datasets Analyzed

- Back to [Table of Contents](#Table-of-Contents)

For this exercise, we will take the datasets you've been introduced to over the last few days and explore: 

- a) different questions that can be answered using these datasets.
- b) the weeds of how we can misinterpret if we do not study the data carefully and understand it well.

## Setup-Coding
- Back to [Table of Contents](#Table-of-Contents)

We first need to get connected to the database. This is as we did in Day 2 in the Intro to SQL session. 

Here, we use the psycopg2 package to interact with the class server. 

### Imports

- back to [Table of Contents](#Table-of-Contents)

In [ ]:
# general use imports
import datetime
import glob
import inspect
import numpy
import os
import six
import warnings

# pandas-related imports
import pandas as pd
import sqlalchemy

# CSV file reading-related imports
import csv

# database interaction imports
import psycopg2
import psycopg2.extras


print( "Imports loaded at " + str( datetime.datetime.now() ) )

### Connecting to Database

- back to [Table of Contents](#Table-of-Contents)

In [ ]:
# schema name - change to your project's schema
output_schema = "<your_project_schema>"

# Use your initials as unique identifier for tables within project schema.
table_unique_prefix = "<your_initials>"

# Database connection properties
db_host = "10.10.2.10"
db_port = -1
db_username = None
db_password = None
db_name = "appliedda"

print( "Database connection properties initialized at " + str( datetime.datetime.now() ) )

### DB Connection  using `psycopg2`

- back to [Table of Contents](#Table-of-Contents)

And then a direct psycopg2 connection and cursor:

In [ ]:
# create psycopg2 connection to Postgresql

# example connect() call that uses all the possible parameters
#pgsql_connection = psycopg2.connect( host = db_host, port = db_port, database = db_name, user = db_username, password = db_password )

# for SQLAlchemy, just needed database name. 
pgsql_connection = psycopg2.connect( host = db_host, database = db_name )

print( "Postgresql connection to database \"" + db_name + "\" created at " + str( datetime.datetime.now() ) )

In [ ]:
# create a cursor that maps column names to values
pgsql_cursor = pgsql_connection.cursor( cursor_factory = psycopg2.extras.DictCursor )

print( "Postgresql cursor for database \"" + db_name + "\" created at " + str( datetime.datetime.now() ) )

In [ ]:
# rollback, in case you need it.
pgsql_connection.rollback()

print( "Postgresql connection for database \"" + db_name + "\" rolled back at " + str( datetime.datetime.now() ) )

## Analysis
back to [Table of Contents](#Table-of-Contents)

### Question: What is the distribution of wages by sex among employees in manufacturing industry in 2010 Q1? 

#### Other interesting questions we can answer using same/similar datasets analyzed
- How many distinct employers in the manufacturing industry in 2010 Q1
- How many distinct employees in the manufacturing infustry in 2010 Q1
- Distribution of heads of household by gender 
- Distribution of heads of household by race



## STEPS

- ### STEP1: Looking in ADRF for the variables of interest
    - Industry Classification
        - NAICS Code- North American Industry Classification System (variable- NAICS, auxiliary_naics)
            *we will use auxiliary naics since it does not have missing values (only 1%); and 0% for 2010*
            Look in ADRF for *2007 North American Industry Classification System (NAICS) Definitions data file*. We have 6 digit NAICS classification code. To subset industry, we will use the first two digits of the longer code. Any code starting with 31, 32 or 33 is specifying manufacturing industry. 
        - SIC- Standard Industry Classification
            *NAICS has replaced the SIC system in 1997.*
    - Employer Identification
        - empr_no, ein, seinunit
    - Year of Interest
        - 2010
       
- ### [STEP2](#STEP-2): Getting/Creating relevant data tables
    - [DATA A](#STEP-2;-DATA-A): Getting a list of all employers in manufacturing industries in 2010. We will use the dataset "ides.il_qcew_employers" for this
    - [DATA B](#STEP-2;-DATA-B): Getting a list of all employees and their wage earnings for first quarter in 2010 from "ides.il_wage"
    - [DATA C](#STEP-2;-DATA-C): Getting a random sample of 10,000 rows from wage data
    - [DATA D](#STEP-2;-DATA-D): Matching DATA A and DATA C
    - [DATA E](#STEP-2;-DATA-E): MATCHING Data D HH_Member data to subset information only for head of households
    - [DATA F](#STEP-2;-DATA-F): Comparing avergae wages across sex

## STEP-2
- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

### STEP-2; DATA-A

Getting a list of all employers in manufacturing industries in 2010. We will use the dataset "ides.il_qcew_employers" for this

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

In [ ]:
# generate SQL
sql_string = "CREATE table if not exists " + output_schema + "." + table_unique_prefix + "naics_2010 as"
sql_string += " SELECT DISTINCT empr_no, seinunit, ein, name_legal, auxiliary_naics "
#chosen the variables of interest
sql_string += " from ides.il_qcew_employers "
sql_string += " where substr(auxiliary_naics,1,2) in ('31', '32', '33')"
#subset the data to contain only those values whose naics value starts from 31, 32 or 33
sql_string += " and year=2010 and quarter=1"
#subset the data for the time period of interest
sql_string += ";"
#this seim-colon is not necessary in psycopg2 but it is widely used across SQL supporting programs. 

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE NAICS_2010 created on " + str( datetime.datetime.now() ) )

#Rollback query below if you need it
#pgsql_connection.rollback()

In [ ]:
#Now that we created our table, let us look at some of the top rows
#generating read SQL
sql_string1 = "select * from " + output_schema +  "." + table_unique_prefix + "naics_2010 limit 10"
#since we gave a limit 10 option in our query, it is not necessary to use the head command below
#(unless you want to see fewer than 10 rows)

# read it
r = pd.read_sql(sql_string1, con=pgsql_connection)
r.head(10)

In [ ]:
#It is likely that you will see that some employers do not have a legal name. 
#While not necessary for this aggregate analysis- this might be a question of interest before proceeding. 


#Let us find how many employers do not have a legal name?

#generating read SQL
sql_string2 = "select count(distinct empr_no) from " + output_schema + "." +  table_unique_prefix + "naics_2010 " 
sql_string2+= " WHERE name_legal in ('nan', '.', '') "
# read it
r = pd.read_sql(sql_string2, con=pgsql_connection)
r



In [ ]:
#It is important to see how many total number of employers are there in the data. 
#This is to take the value of missing legal names into 'statistical perspective'

#generating read SQL
sql_string3 = "select count(distinct empr_no) from " + output_schema + "." + table_unique_prefix + "naics_2010 "

# read it
r = pd.read_sql(sql_string3, con=pgsql_connection)
r


#### Missing Values

- XXX of our data has missing legal names
- A good rule of thumb is that we can drop data with missing values if missing values is less than 5% of the data. 
    - However, in this case: 
     - a) our variable of interest 'name_legal' is not important to us
     - b) and missing values are very high

**So we will not drop the data**

*An alternative and probably better way to see if the employers with no legal names can be dropped is to see the below: 
    1. what percentage of employees work in these firms
    2. how many wages are earned by the employees working with these employers
This will again be only necessary if we care about this variable to begin with. 
Since this is an aggregate level study- we do not care about individual employers or their names. 
However, it could have been a variable of interest in some other study*

- We could have done the same analysis for other variables of interest also such as: 
    - seinunit or ein
    - naics (but it will not be missing since we subsetted our data for manufacturing codes initially)

### USING A CONDITION STATEMENT IN SQL

##### Of interest: If an employer has a legal name or not
#### SQL QUERIES USED: 
    * CASE WHEN
    * GROUP BY
    * SUM
    * OVER
    * DISTINCT


In [ ]:
##WE COULD HAVE DONE THE SAME THING IN ONE QUERY ALSO
#Calculating the percentage of employers having a legal name vs the ones not

#CREATING AN ID with a CONDITION STATEMENT (CALLED CASE in SQL)
sql_string3 = "select case when name_legal in ('nan', '.', '') then 1 else 2 end as name_type, "
#Creating an ID with a value of 1 if name_legal is missing and a value of 2 if not missing
sql_string3 += "count(distinct empr_no), "
#Counting the distinct number of employers associated with both IDs
sql_string3 += "count(distinct empr_no)/(sum(count(distinct empr_no)) over()) PER "
#Counting the percentage of values in both IDs
sql_string3 += "from " + output_schema + "." + table_unique_prefix + "naics_2010 " 
#SPECIFYING THE BASE TABLE
sql_string3 += " group by name_type "
#GROUPING THE SUM for both ID

# read it
r = pd.read_sql(sql_string3, con=pgsql_connection)
r



### STEP-2; DATA-B

Getting a list of all employees and their wage earnings for first quarter in 2010 from "ides.il_wage"

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

In [ ]:
#STEP 2- DATA B
#DATA B: Getting a list of all employees and their wage earnings for first quarter in 2010 from "ides.il_wage"

# generate SQL
sql_string = "CREATE table if not exists " + output_schema + "." + table_unique_prefix + "employee_2010 as"
sql_string += " SELECT DISTINCT ssn, empr_no, seinunit, ein, wage, hours, weeks"
sql_string += " from ides.il_wage "
sql_string += " where year=2010 and quarter=1"
sql_string += ";"


# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE employee_2010 created on " + str( datetime.datetime.now() ) )

#pgsql_cursor.execute("drop table data_prep.naics_2010")
# pgsql_connection.commit()

#pgsql_connection.rollback()

In [ ]:
#looking at our data
#generating read SQL
sql_string2 = "select * from " + output_schema + "." + table_unique_prefix + "employee_2010"
#sql_string2 += " limit 10"

# read it
r = pd.read_sql(sql_string2, con=pgsql_connection)
r.head(5)
#You can test the time taken to run this query by using the limit option vs not using. 
#While the limit option will lead to the query reading only 10 rows, 
#not using it will lead the query reading all of the rows- yet, reporting only 5 

#It is always better to use the limit option if we know we are not interested in reading the whole data

In [ ]:
#looking at #employees
#generating read SQL
sql_string3 = "select count(distinct ssn) from " + output_schema + "." + table_unique_prefix + "employee_2010"
# read it
r = pd.read_sql(sql_string3, con=pgsql_connection)
r


In [ ]:
sql_string3 = "select count(*) from " + output_schema + "." + table_unique_prefix + "employee_2010"
r = pd.read_sql(sql_string3, con=pgsql_connection)
r


In [ ]:
#looking at employees with non-missing wages

sql_string4 = "select count(distinct ssn) from " + output_schema + "." + table_unique_prefix + "employee_2010"+ " where wage>0"
r = pd.read_sql(sql_string4, con=pgsql_connection)
r


### STEP-2; DATA-C

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

#### SQL QUERY- taking a random sample

We want to take a random subset of 10,000 employees to shorten the running time of this query. 


In [ ]:
#Taking a random sample of employees


sql_string = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "emp_2010_random as"
sql_string += " select * from " + output_schema + "." + table_unique_prefix + "employee_2010"
sql_string += " ORDER BY RANDOM()  "
sql_string += " LIMIT 10000"


#we use distinct function so we only get unique rows
# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE emp_2010_random created on " + str( datetime.datetime.now() ) )
#pgsql_connection.rollback()


In [ ]:
#TESTING IF THIS IS INDEED 1000 rows


sql_string = "select count(*) from " + output_schema + "." + table_unique_prefix + "emp_2010_random"

#we use distinct function so we only get unique rows
# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

r = pd.read_sql(sql_string, con=pgsql_connection)
r
#pgsql_connection.rollback()


In [ ]:
#pgsql_connection.rollback()


### STEP-2; DATA-D

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

#### SQL QUERY-  Matching the Data
- LEFT JOIN

In [ ]:
#STEP 2- DATA D
#DATA D: MATCHING DATA A & DATA B--- to get a list of employees who are working in manufacturing industry

sql_string = "CREATE TABLE if not exists " + output_schema + "." +  table_unique_prefix + "emp_manuf as"
sql_string += " select distinct a.name_legal, a.auxiliary_naics, b.* from " + output_schema + "." + table_unique_prefix + "naics_2010 as a"
sql_string += " LEFT JOIN " + output_schema + "." + table_unique_prefix + "emp_2010_random as b"  
sql_string += " on a.empr_no=b.empr_no "
sql_string += " and a.seinunit=b.seinunit "
sql_string += " and a.ein=b.ein"

#we use distinct function so we only get unique rows
# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE emp_manuf created on " + str( datetime.datetime.now() ) )


In [ ]:
# sql_string = "select * from " + output_schema + "." + table_unique_prefix + "EMP_MANUF"+ " LIMIT 1"
# r = pd.read_sql(sql_string, con=pgsql_connection)
# r

sql_string = "select count(*), count(distinct ssn) from " + output_schema + "." + table_unique_prefix + "EMP_MANUF"
r = pd.read_sql(sql_string, con=pgsql_connection)
r




### STEP-2; DATA-E-1

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

#### SQL QUERY-  
- CASE WHEN


In [ ]:
#STEP 2- DATA E-1
#DATA E-1: Preparing the HH_MEMBER data

sql_string = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "hh_mem0 as"
sql_string += " select distinct ssn_hash, rootrace, ssnind ,sex"
sql_string += " from idhs.hh_member "  

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE hh_mem0 created on " + str( datetime.datetime.now() ) )


In [ ]:
# Counting number of distinct SSN vs number of rows in this data

sql_string = "select count(*), count(distinct ssn_hash) from " + output_schema + "." + table_unique_prefix + "hh_mem0"
r = pd.read_sql(sql_string, con=pgsql_connection)
r



In [ ]:
#pgsql_connection.rollback()

In [ ]:



sql_string  = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "hh_mem1 as"
sql_string += " select ssn_hash,  count(*) as cnt "
sql_string += "from " + output_schema + "." + table_unique_prefix + "hh_mem0 group by ssn_hash"
sql_string += " order by cnt desc "

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()


In [ ]:
#reading the newly created table
sql_string="select * from " + output_schema + "."  + ind + "hh_mem1 limit 5"
s = pd.read_sql(sql_string, con=pgsql_connection)
s

In [ ]:
#Creating a sub table which has unique rows for each ssn
sql_string  = "Select cnt, count(*) as cnt,"
sql_string += " 100*(count(*)/(sum(count(*)) over())) as per" 
sql_string += " from "+ output_schema + "." + table_unique_prefix + "hh_mem1"
sql_string += " group by cnt"
s = pd.read_sql(sql_string, con=pgsql_connection)
s



In [ ]:
#STEP 2- DATA E-1
#DATA E-1: Creating the subsetted data table for hh_member which is cleaned to maintain one unique row per individual

sql_string = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "hh_mem2 as"
sql_string += " select distinct ssn_hash, rootrace, ssnind ,sex"
sql_string += " from " + output_schema + "." + table_unique_prefix + "hh_mem0 "  
sql_string += " where ssn_hash in "
sql_string += " (select ssn_hash from " + output_schema + "." + table_unique_prefix + "hh_mem1"
sql_string += " where cnt=1) "

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE hh_mem2 created on " + str( datetime.datetime.now() ) )


In [ ]:
#pgsql_connection.rollback()

### STEP-2; DATA-E-2

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

#### SQL QUERY-  Matching the Data
- INNER JOIN

In [ ]:
#STEP 2- DATA E-2
#DATA E: MATCHING DATA WITH HH_MEMBERS
#WE only want information for head of households

#You can use the base data hh_mem0 as well which has not been cleaned. 
#The only difference is that then you will not be able to use the summary measure 'avg' as explained further
#keep reading!

match_table= "hh_mem2"
#match_table= "hh_mem0"


sql_string = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "emp_manuf" + "_" +match_table+ " as"
sql_string += " select a.*, b.ssn_hash, b.rootrace, b.ssnind , b.sex"
sql_string += " from " + output_schema + "." + table_unique_prefix + "emp_manuf as a"  
sql_string += " inner join "  + output_schema + "." + table_unique_prefix + match_table  +  " as b"  
sql_string += " on a.ssn=b.ssn_hash"

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE emp_manuf_hh created on " + str( datetime.datetime.now() ) )


In [ ]:
#number of employees
sql_string = "select count(*) as cnt_rows, count(distinct ssn) as cnt_ind from " + output_schema + "." + table_unique_prefix + "emp_manuf" + "_" +match_table
r = pd.read_sql(sql_string, con=pgsql_connection)
r

#If you match the above data with hh_mem0 (not cleaned), you will get ## records for ## employees. 
#If you match the above data with hh_mem2 (cleaned), you will get ## records for ##-X employees. 

# sql_string = "select * from " + output_schema + "." + ind + "EMP_MANUF_HH LIMIT 2"
# s = pd.read_sql(sql_string, con=pgsql_connection)
# s



### STEP-2; DATA-F

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

#### Getting aggregate measures by SEX

In [ ]:
#STEP 2- DATA F
#DATA F: Aggregating data - by sex
#WE only want information for head of households


match_table= "hh_mem2"
#match_table= "hh_mem0"

sql_string = "CREATE TABLE if not exists " + output_schema + "." + table_unique_prefix + "emp_manuf_hh_Sex as"
sql_string += " select sex, count(distinct ssn) as cnt, sum(wage) as sum, "
sql_string += "100*(count(distinct ssn)/sum(count(distinct ssn)) over()) as per_cnt, "
sql_string += "100*(sum(wage)/ (sum(sum(wage)) over())) as per_wage"

sql_string += " from " + output_schema + "." + table_unique_prefix + "emp_manuf_" + match_table + " group by sex;"


#group by ssn, rootrace, sex
# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()

print( "TABLE emp_manuf_hh created on" + str( datetime.datetime.now() ) )


In [ ]:
#Reading the data

#sql_string = "DROP TABLE " + output_schema + "." + ind + "EMP_MANUF_HH_Sex "

sql_string = "select * from " + output_schema + "." + table_unique_prefix + "EMP_MANUF_HH_Sex "
s = pd.read_sql(sql_string, con=pgsql_connection)
s



In [ ]:

#Calculating average wages for each gender group

sql_string = "select sex, sum/cnt as avg_wage  from " + output_schema + "." + table_unique_prefix + "EMP_MANUF_HH_Sex  "
s = pd.read_sql(sql_string, con=pgsql_connection)
s



### Using summary queries in SQL

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
- back to [STEPS](#STEPS)

#### When to use these instead
Instead of actually calculating the average, we can rely on some simple SQL queries to compute mean measures. 

- Since we cleaned our data to have one row per individual- we can also rely on a simple SQL query 'average' to calculate average wages by any variable of interest. 
- However, if we had not cleaned our data-- then it will have to be necessary to actually 'calculate'- since SQL will take the number of rows- and not the number of distinct individuals (ssn) to calculate averages. 


In [ ]:
#ALTERNATIVE and QUICKER WAY TO DO THIS

sql_string = "select sex, count(distinct ssn) as ind_cnt, "
sql_string += "avg(wage)  from " + output_schema + "." + table_unique_prefix + "EMP_MANUF_HH group by sex "
s = pd.read_sql(sql_string, con=pgsql_connection)
s

In [ ]:
#OTHER SUMMARY MEASURES

sql_string = "select rootrace, count(distinct ssn) as ind_cnt, avg(wage) avg from " + output_schema + "." + table_unique_prefix + "EMP_MANUF_HH group by rootrace order by avg "
s = pd.read_sql(sql_string, con=pgsql_connection)
s

#  1=White, not of Hispanic origin, 
#  2=Black, not of Hispanic origin, 
#  3=American Indian or Alaskan Native, 
#  6=Hispanic (includes Mexican, Puerto Rican, Cuban, and Other South American), 
#  7=Asian or Pacific Islander (includes Indo-Chinese)

# Getting to know the IDHS database

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)

In [ ]:
#Looking at the variables of interest

# generate SQL
sql_string = " SELECT * "
sql_string += " FROM idhs.hh_member"
sql_string += " LIMIT 10"
sql_string += ";"

# read it
r = pd.read_sql(sql_string, con=pgsql_connection)
r.head(1)

## IDHS Database
## HH_Member table Descrition

If you go to the ADRF explorer, and read documentation, you will see what the different variables mean. 
Here, we paste the description of the variables we will be using for this notebook. 

1. sex- 1 for male, 2 for female
2. rootrace        
    - 1=White, not of Hispanic origin, 
    - 2=Black, not of Hispanic origin, 
    - 3=American Indian or Alaskan Native, 
    - 6=Hispanic (includes Mexican, Puerto Rican, Cuban, and Other South American), 
    - 7=Asian or Pacific Islander (includes Indo-Chinese)
3. ssn_hash- hashed SSN
4. ssnind- Indicates the validity of the recipient social security number. 
    - J=Validity unknown, 
    - K=SSA not in SSA file, 
    - L=Sex code does not match SSA file,
    - M=DOB does not match SSA file, 
    - N=DOB and sex code do not match SSA file, 
    - O=Name does not match SSA file (sex code and DOB not checked), 
    - P=SS5 form is pending with SSA, R=SS5 form is pending with enumeration control, 
    - V=SS Administration has verified SSN

In [ ]:
# sql_string1 = "SELECT sex, count( *) as cnt, count(*)/sum(count(*)) over() as PER from idhs.hh_member group by sex"
# r1 = pd.read_sql(sql_string1, con=pgsql_connection)
# print(r1)

sql_string2 = "SELECT ssnind, count( *) as cnt, 100*(count(*)/sum(count(*)) over()) as PER from idhs.hh_member group by ssnind"
r2 = pd.read_sql(sql_string2, con=pgsql_connection)
print(r2)

### Printing by INDEX

- back to [Table of Contents](#Table-of-Contents)
- back to [Analysis](#Analysis)
    
We can also print the results of our SQL queries by Column INDEX. 



In [ ]:
# Printing by INDEX

# generate SQL
sql_string = " SELECT *"
sql_string += " FROM idhs.hh_member"
sql_string += " LIMIT 10"
sql_string += ";"

# execute it.
pgsql_cursor.execute(sql_string)
pgsql_connection.commit()
results=pgsql_cursor.fetchall()

#PRINTING BY INDEX

print "\nShow me the databases:\n"
for row in results: 
    print row[1]
   #print "", row[0]

### Sample Queries

- Count of Male & Females within head of household data
- Count of Individuals with no wages in a quarter
- The maximum wage in a particular industry in a quarter
   

In [ ]:
#Count of Male & Females within head of household data

# generate SQL
sql_string = " SELECT sex, count(distinct ssn_hash) as cnt "
sql_string += " FROM idhs.hh_member"
sql_string += " group by sex"
sql_string += ";"

# execute it.
#pgsql_cursor.execute(sql_string)
#pgsql_connection.commit()

r2 = pd.read_sql(sql_string, con=pgsql_connection)
print(r2)

In [ ]:
#Count of Individuals with no wages in a quarter

# **LONG QUERY- DO NOT RUN WHILE IN CLASS**
year='2010'
quarter='2'


# generate SQL
sql_string = " SELECT count(distinct ssn) as cnt "
sql_string += " FROM ides.il_wage"
sql_string += " where year=" + year +"and quarter= " + quarter
sql_string += " and wage=0"

# execute it.
#pgsql_cursor.execute(sql_string)
#pgsql_connection.commit()

r2 = pd.read_sql(sql_string, con=pgsql_connection)
print(r2)

In [ ]:
#The maximum wage and min wages in a particular time period


# **LONG QUERY- DO NOT RUN WHILE IN CLASS**
year='2010'
quarter='2'



# generate SQL
sql_string = " SELECT min(wage) as min, max(wage) as max "
sql_string += " FROM ides.il_wage"
sql_string += " where year=" + year +"and quarter= " + quarter
#sql_string += " and wage>100"

# execute it.
#pgsql_cursor.execute(sql_string)
#pgsql_connection.commit()

r2 = pd.read_sql(sql_string, con=pgsql_connection)
print(r2)

In [ ]:
# Close Connection and cursor
pgsql_cursor.close()
pgsql_connection.close()

print( "psycopg2 cursor and connection closed at " + str( datetime.datetime.now() ) )